In [1]:
import sys

import matplotlib.pyplot as plt
import PIL
from PIL import Image
import json

import torch
import torchvision
import torchvision.transforms as T
# pip3 install timm==0.5.4 
from timm import create_model

/Users/xinyichen/opt/anaconda3/envs/convnext/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import timm
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [3]:
# model_name = "convnext_xlarge_in22ft1k"
model_name = "resnet50"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device = ", device)
# create a ConvNeXt model : https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/convnext.py
model = create_model(model_name, pretrained=True).to(device)

device =  cpu


In [4]:
# Define transforms for test
from timm.data.constants import \
    IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

NORMALIZE_MEAN = IMAGENET_DEFAULT_MEAN
NORMALIZE_STD = IMAGENET_DEFAULT_STD
SIZE = 256

# Here we resize smaller edge to 256, no center cropping
transforms = [
              T.Resize(SIZE, interpolation=T.InterpolationMode.BICUBIC),
              T.ToTensor(),
              T.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
              ]

transforms = T.Compose(transforms)

### Test on counterfactural images

In [7]:
import pandas as pd
import os
from tqdm import tqdm

path = '../../2022_05_15_17_RUN_NAME_trunc_0.5/labels.csv'
df = pd.read_csv(path, index_col=0)
df = df.set_index('im_name')
shape_cls = df['shape_cls']
texture_cls = df['texture_cls']
bg_cls = df['bg_cls']

In [8]:
img_dir = '../../2022_05_15_17_RUN_NAME_trunc_0.5/ims'
idx_to_tag = {0:'shape_cls', 1:'texture_cls', 
              2:'bg_cls', 3:'wrong'}
# sp_cnt = tt_cnt = bg_cnt = wrong_cnt
cnt = [0, 0, 0, 0]
for img_name in tqdm(shape_cls.keys()):
    img_path = os.path.join(img_dir, img_name+'_x_gen.jpg')
    img = PIL.Image.open(img_path)
    img_tensor = transforms(img).unsqueeze(0).to(device)
    output = torch.softmax(model(img_tensor), dim=1)
    top5 = torch.topk(output, k=5)
    top5_prob = top5.values[0]
    top5_indices = top5.indices[0]
    predict = top5_indices[0]
    target = [shape_cls[img_name], texture_cls[img_name], bg_cls[img_name]]
    if predict in target:
        cnt[target.index(predict)] += 1
    else:
        cnt[3] += 1

print(cnt)

100%|██████████| 100/100 [00:26<00:00,  3.77it/s]

[0, 8, 0, 92]


In [10]:
import numpy as np

results = (np.array(cnt) / len(shape_cls)).tolist()
for i, r in enumerate(results):
    print(idx_to_tag[i], r, end = ',')

shape_cls 0.0,texture_cls 0.08,bg_cls 0.0,wrong 0.92,

### Model results
convnext_xlarge_in22ft1k: shape_cls 0.01,texture_cls 0.52,bg_cls 0.05,wrong 0.42

resnet50: shape_cls 0.0,texture_cls 0.08,bg_cls 0.0,wrong 0.92,
